In [80]:
# Importing libraries

import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re 

In [83]:
# Creating a function to get data in HTML format from website.

def getHTML(url):
    htmlcontent = urlopen(url)
    beautify = BeautifulSoup(htmlcontent, 'html.parser')
    return beautify

In [84]:
# Using a function 'getHTML' to get data from Wikipedia

uscities = getHTML('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')


In [85]:
#Get the whole table as well as links for the each city page.

links_table = uscities.find('table', {'class': 'wikitable sortable'})
rows = links_table.find_all('tr')

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')    
    if len(tds) != 0:
        city_link = tds[1].find('a')
        city_info = [td.text.replace('\n', '').replace('\xa0', '').replace('\ufeff', '') for td in tds]
        #print(city_link.get('href'))
        print(city_info)

['1', 'New York City[d]', 'New York', '8,398,748', '8,175,133', '+2.74%', '301.5sqmi', '780.9km2', '28,317/sqmi', '10,933/km2', '40°39′49″N 73°56′19″W / 40.6635°N 73.9387°W / 40.6635; -73.9387 (1 New York City)']
['2', 'Los Angeles', 'California', '3,990,456', '3,792,621', '+5.22%', '468.7sqmi', '1,213.9km2', '8,484/sqmi', '3,276/km2', '34°01′10″N 118°24′39″W / 34.0194°N 118.4108°W / 34.0194; -118.4108 (2 Los Angeles)']
['3', 'Chicago', 'Illinois', '2,705,994', '2,695,598', '+0.39%', '227.3sqmi', '588.7km2', '11,900/sqmi', '4,600/km2', '41°50′15″N 87°40′54″W / 41.8376°N 87.6818°W / 41.8376; -87.6818 (3 Chicago)']
['4', 'Houston[3]', 'Texas', '2,325,502', '2,100,263', '+10.72%', '637.5sqmi', '1,651.1km2', '3,613/sqmi', '1,395/km2', '29°47′12″N 95°23′27″W / 29.7866°N 95.3909°W / 29.7866; -95.3909 (4 Houston)']
['5', 'Phoenix', 'Arizona', '1,660,272', '1,445,632', '+14.85%', '517.6sqmi', '1,340.6km2', '3,120/sqmi', '1,200/km2', '33°34′20″N 112°05′24″W / 33.5722°N 112.0901°W / 33.5722; -11

In [86]:
# Defining a function to get data from individual city pages. Here we are extacting only name of 'Mayor' for each city.

def getAdditionalDetails(url):
    try:
        city_page = getHTML('https://en.wikipedia.org' + url) 
        table = city_page.find('table', {'class': 'infobox geography vcard'})
        additional_details = []
        read_content = False
        for tr in table.find_all('tr'):
            if (tr.get('class') == ['mergedtoprow'] and not read_content):
                link = tr.find('th')
                if (link and (link.get_text().strip() == 'Government')):
                    read_content = True
                    
            if ((tr.get('class') == ['mergedrow']) and read_content):               
                if (tr.find('th').get_text().strip() == '•\xa0Mayor'):
                        additional_details.append(tr.find('td').get_text().strip('\n')) 
                                   
        return additional_details
    except Exception as error:
        print('Error occured: {}'.format(error))
        return []


In [108]:
data_final = []
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')    
    if len(tds) != 0:
        city_link = tds[1].find('a')
        city_info = [td.text.replace('\n', '').replace('\xa0', '').replace('\ufeff', '') for td in tds]
        city_details = getAdditionalDetails(city_link.get('href'))
        city_info += city_details
        data_final.append(city_info)
        
df = pd.DataFrame(data_final)

KeyboardInterrupt: 

In [109]:
# Define column headings

headers = rows[0].find_all('th')
headers = [header.get_text().strip('\n') for header in headers]
headers += ['Mayor']
headers.insert(7,'2016 Land Area(km2)')
headers.insert(9,'2016 Population Density(km2)')
#print(headers)
df.columns = headers
df.rename(columns={'State[c]': 'State'}, inplace = True)
df.rename(columns={'2016 land area': '2016 Land area(sqmi)'}, inplace = True)
df.rename(columns={'2016 population density': '2016 Population Density(sqmi)'}, inplace = True)
#print(df)

In [110]:
for column in df.columns:
    df[column] = df[column].str.replace(r"\(.*\)", "")
    df[column] = df[column].str.replace(r"\[.*\]", "")
    df['Change'] = df['Change'].str.strip('%')
    df['2016 Land area(sqmi)'] = df['2016 Land area(sqmi)'].str.strip("/sqmi")
    df['2016 Land Area(km2)'] = df['2016 Land Area(km2)'].str.strip("/km")
    df['2016 Population Density(sqmi)'] = df['2016 Population Density(sqmi)'].str.strip("/sqmi")
    df['2016 Population Density(km2)'] = df['2016 Population Density(km2)'].str.strip("/km")
    
#The above print statement will display give the final dataset.    
    print(df)
    
df.to_csv("Final_Dataset.csv", index = False)

    2018rank              City                 State 2018estimate 2010Census  \
0          1     New York City              New York    8,398,748  8,175,133   
1          2       Los Angeles            California    3,990,456  3,792,621   
2          3           Chicago              Illinois    2,705,994  2,695,598   
3          4           Houston                 Texas    2,325,502  2,100,263   
4          5           Phoenix               Arizona    1,660,272  1,445,632   
5          6      Philadelphia          Pennsylvania    1,584,138  1,526,006   
6          7       San Antonio                 Texas    1,532,233  1,327,407   
7          8         San Diego            California    1,425,976  1,307,402   
8          9            Dallas                 Texas    1,345,047  1,197,816   
9         10          San Jose            California    1,030,119    945,942   
10        11            Austin                 Texas      964,254    790,390   
11        12      Jacksonville          